In [2]:
import sys
sys.path.insert(0, '../scripts')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import etl_new as etl

In [2]:
!ls ../source/csv

WPP2019_LifeTable.csv
WPP2019_Period_Indicators_Medium.csv
WPP2019_Period_Indicators_OtherVariants.csv
WPP2019_PopulationByAgeSex_Medium.csv
WPP2019_PopulationByAgeSex_OtherVariants.csv
WPP2019_PopulationBySingleAgeSex.csv
WPP2019_TotalPopulationBySex.csv


In [3]:
popbyagesex = pd.read_csv('../source/csv/WPP2019_PopulationBySingleAgeSex.csv', encoding='latin1')

In [4]:
popbyagesex.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0,0,1,139.669,154.913,294.581
1,4,Afghanistan,2,Medium,1950,1950.5,1,1,1,131.916,141.851,273.767
2,4,Afghanistan,2,Medium,1950,1950.5,2,2,1,125.127,130.632,255.759
3,4,Afghanistan,2,Medium,1950,1950.5,3,3,1,119.220,121.097,240.317
4,4,Afghanistan,2,Medium,1950,1950.5,4,4,1,114.112,113.085,227.198


In [7]:
from ddf_utils.str import to_concept_id

In [6]:
popbyagesex['AgeGrpSpan'].unique()

array([ 1, -1])

In [7]:
popbyagesex[popbyagesex['AgeGrpSpan'] == -1]['AgeGrp'].unique()

array([100])

In [8]:
agegroup = popbyagesex[['AgeGrp']].drop_duplicates()
agegroup['age1yearinterval'] = agegroup['AgeGrp']
agegroup.columns = ['name', 'age1yearinterval']

In [9]:
agegroup.head()

,name,age1yearinterval
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [10]:
agegroup[['age1yearinterval', 'name']].to_csv('../../ddf--entities--age1yearinterval.csv', index=False)

In [11]:
variants = popbyagesex[['VarID', 'Variant']].drop_duplicates()

In [12]:
variants

,VarID,Variant
0,2,Medium


In [13]:
popbyagesex.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'AgeGrp',
       'AgeGrpStart', 'AgeGrpSpan', 'PopMale', 'PopFemale', 'PopTotal'],
      dtype='object')

In [14]:
dp = popbyagesex[['LocID', 'Time', 'AgeGrp', 'PopMale', 'PopFemale', 'PopTotal']].copy()
# dp['AgeGrp'] = dp['AgeGrp'].map(to_concept_id)
dp.columns = ['location', 'time', 'age1yearinterval', 'pop_male', 'pop_female', 'pop_total']

In [15]:
dp = dp.set_index(['location', 'time', 'age1yearinterval'])

In [16]:
dfs = []
for c in dp:
    sex = c.split('_')[-1]
    df = dp[[c]].copy()
    df.columns = ['population']
    df['sex'] = sex
    df = df.set_index('sex', append=True)
    dfs.append(df)

In [17]:
dp_merged = pd.concat(dfs, sort=False)

In [18]:
dp_merged.head()

population
location time age1yearinterval sex             
4        1950 0                male     139.669
              1                male     131.916
              2                male     125.127
              3                male     119.220
              4                male     114.112

In [19]:
dp_merged.query('location == 900').head()

population
location time age1yearinterval sex             
900      1950 0                male   39854.036
              1                male   36612.375
              2                male   33956.483
              3                male   31828.068
              4                male   30168.870

In [9]:
location_metadata = pd.read_excel('../source/WPP2019_F01_LOCATIONS.XLSX', skiprows=16)

In [21]:
location_metadata.columns

Index(['Index', 'Region, subregion, country or area*', 'Notes',
       'Location code', 'ISO3 Alpha-code', 'Code', 'Name',
       'Parent Location code', 'Code.1', 'Code.2', 'Name.1', 'Code.3',
       'Name.2', 'Code.4', 'Name.3', 'Code.5', 'Name.4',
       'More developed regions\n901', 'Less developed regions\n902',
       'Least developed countries\n941',
       'Less developed regions, excluding least developed countries\n934',
       'Less developed regions, excluding China\n948',
       'Land-Locked Developing Countries (LLDC)\n1636',
       'Small Island Developing States (SIDS)\n1637',
       'High-income Countries\n1503', 'Middle-income Countries\n1517',
       'Upper-middle-income Countries\n1502',
       'Lower-middle-income Countries\n1501', 'Low-income Countries\n1500',
       'No income group available\n1518', 'Max(HIV) for males',
       'Max(HIV) for females', 'Max(HIV) for both sexes',
       'Year for max(HIV) for both sexes',
       'Estimates of mortality age patter

In [10]:
loc = location_metadata[['Region, subregion, country or area*',
                         'Location code',
                         'ISO3 Alpha-code',
                         'Parent Location code',
                         'Code',
                         'Name']].copy()

In [23]:
loc.head()

,"Region, subregion, country or area*",Location code,ISO3 Alpha-code,Parent Location code,Code,Name
0,WORLD,900,NaN,0,NaN,NaN
1,UN development groups,1803,NaN,900,25.0,Label/Separator
2,More developed regions,901,NaN,1803,5.0,Development group
3,Less developed regions,902,NaN,1803,5.0,Development group
4,Least developed countries,941,NaN,902,5.0,Development group


In [11]:
loc[loc['Name'] == 'Country/Area'].head()

,"Region, subregion, country or area*",Location code,ISO3 Alpha-code,Parent Location code,Code,Name
26,Burundi,108,BDI,910,4.0,Country/Area
27,Comoros,174,COM,910,4.0,Country/Area
28,Djibouti,262,DJI,910,4.0,Country/Area
29,Eritrea,232,ERI,910,4.0,Country/Area
30,Ethiopia,231,ETH,910,4.0,Country/Area


In [12]:
countries = loc[loc['Name'] == 'Country/Area'].copy()
countries.columns = ['name', 'country', 'iso3', 'parent', 'code', 'type']
countries = countries[['country', 'name', 'iso3']]

In [26]:
countries.head()

,country,name,iso3
26,108,Burundi,BDI
27,174,Comoros,COM
28,262,Djibouti,DJI
29,232,Eritrea,ERI
30,231,Ethiopia,ETH


In [13]:
countries['is--country'] = 'TRUE'

In [42]:
# countries.to_csv('../../ddf--entities--geo--country.csv', index=False)
countries[['country', 'name', 'iso3']].to_csv('../../ddf--entities--country.csv', index=False)

In [28]:
dp_country = dp_merged.reset_index()

In [32]:
dp_country = dp_country.rename(columns={'location': 'country', 'sex': 'gender'})

In [33]:
dp_country.head()

,country,time,age1yearinterval,gender,population
0,4,1950,0,male,139.669
1,4,1950,1,male,131.916
2,4,1950,2,male,125.127
3,4,1950,3,male,119.220
4,4,1950,4,male,114.112


In [39]:
dp_total = dp_country[dp_country.gender == 'total']
dp_total = dp_total[['country', 'time', 'age1yearinterval', 'population']]

for c in countries['country'].unique():
    df = dp_total[dp_total['country'] == c]
    if not df.empty:
        df.to_csv(f'../../populations--by--country--time--age1yearinterval/ddf--datapoints--population--by--country-{c}--time--age1yearinterval.csv'
                  , index=False)

In [40]:
dp_genders = dp_country[dp_country.gender.isin(['male', 'female'])]

for c in countries['country'].unique():
    df = dp_genders[dp_genders['country'] == c]
    if not df.empty:
        df.to_csv(
            f'../../populations--by--country--time--age1yearinterval--gender/ddf--datapoints--population--by--country-{c}--time--age1yearinterval--gender.csv'
            , index=False)

In [35]:
countries.columns

Index(['country', 'name', 'iso3'], dtype='object')

In [14]:
peroidInd = pd.read_csv('../source/csv/WPP2019_Period_Indicators_Medium.csv', encoding='latin1')

In [44]:
peroidInd.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,TFR,NRR,CBR,Births,...,CDR,Deaths,DeathsMale,DeathsFemale,CNMR,NetMigrations,GrowthRate,NatIncr,SRB,MAC
0,4,Afghanistan,2,Medium,1950-1955,1953,7.45,1.636,50.314,2015.476,...,36.862,1476.605,794.020,682.585,-0.499,-20.0,1.296,13.452,1.06,29.835
1,4,Afghanistan,2,Medium,1955-1960,1958,7.45,1.765,50.998,2201.589,...,33.718,1455.605,783.110,672.495,-0.463,-20.0,1.683,17.280,1.06,29.835
2,4,Afghanistan,2,Medium,1960-1965,1963,7.45,1.899,51.477,2439.130,...,30.808,1459.784,777.437,682.347,-0.422,-20.0,2.026,20.669,1.06,29.835
3,4,Afghanistan,2,Medium,1965-1970,1968,7.45,2.017,51.646,2728.221,...,28.223,1490.896,790.745,700.151,-0.379,-20.0,2.307,23.423,1.06,29.835
4,4,Afghanistan,2,Medium,1970-1975,1973,7.45,2.141,51.234,3056.451,...,25.495,1520.938,802.414,718.524,-0.335,-20.0,2.544,25.739,1.06,29.835


In [2]:
pi_concepts = {
    "TFR          ": "Total fertility (live births per woman)",
    "NRR          ": "Net reproduction rate (surviving daughters per woman)",
    "CBR          ": "Crude birth rate (births per 1,000 population)",
    "Births       ": "Number of births, both sexes combined (thousands)",
    "LEx          ": "Life expectancy at birth for both sexes combined (years)",
    "IMR          ": "Infant mortality rate, q(1), for both sexes combined (infant deaths per 1,000 live births)",
    "Q5           ": "Under-five mortality, 5q0, for both sexes combined (deaths under age five per 1,000 live births)",
    "CDR          ": "Crude death rate (deaths per 1,000 population)",
    "Deaths       ": "Number of deaths, both sexes combined (thousands)",
    "CNMR         ": "Net migration rate (per 1,000 population)",
    "NetMigrations": "Net number of migrants, both sexes combined (thousands)",
    "GrowthRate   ": "Average annual rate of population change (percentage)",
    "NatIncr      ": "Rate of natural increase (per 1,000 population)",
    "SRB          ": "Sex ratio at birth (male births per female births)",
    "MAC          ": "Female mean age of childbearing (years)"
}

In [3]:
pi_concepts = dict([(k.strip(), v) for k, v in pi_concepts.items()])

In [4]:
pi_concepts

{'TFR': 'Total fertility (live births per woman)',
 'NRR': 'Net reproduction rate (surviving daughters per woman)',
 'CBR': 'Crude birth rate (births per 1,000 population)',
 'Births': 'Number of births, both sexes combined (thousands)',
 'LEx': 'Life expectancy at birth for both sexes combined (years)',
 'IMR': 'Infant mortality rate, q(1), for both sexes combined (infant deaths per 1,000 live births)',
 'Q5': 'Under-five mortality, 5q0, for both sexes combined (deaths under age five per 1,000 live births)',
 'CDR': 'Crude death rate (deaths per 1,000 population)',
 'Deaths': 'Number of deaths, both sexes combined (thousands)',
 'CNMR': 'Net migration rate (per 1,000 population)',
 'NetMigrations': 'Net number of migrants, both sexes combined (thousands)',
 'GrowthRate': 'Average annual rate of population change (percentage)',
 'NatIncr': 'Rate of natural increase (per 1,000 population)',
 'SRB': 'Sex ratio at birth (male births per female births)',
 'MAC': 'Female mean age of childbe

In [8]:
c = pd.DataFrame.from_dict(pi_concepts, orient='index')
c.index = c.index.map(to_concept_id)
c['concept_type'] = 'measure'
c['domain'] = ''

print(c.to_csv())

,0,concept_type,domain
tfr,Total fertility (live births per woman),measure,
nrr,Net reproduction rate (surviving daughters per woman),measure,
cbr,"Crude birth rate (births per 1,000 population)",measure,
births,"Number of births, both sexes combined (thousands)",measure,
lex,Life expectancy at birth for both sexes combined (years),measure,
imr,"Infant mortality rate, q(1), for both sexes combined (infant deaths per 1,000 live births)",measure,
q5,"Under-five mortality, 5q0, for both sexes combined (deaths under age five per 1,000 live births)",measure,
cdr,"Crude death rate (deaths per 1,000 population)",measure,
deaths,"Number of deaths, both sexes combined (thousands)",measure,
cnmr,"Net migration rate (per 1,000 population)",measure,
netmigrations,"Net number of migrants, both sexes combined (thousands)",measure,
growthrate,Average annual rate of population change (percentage),measure,
natincr,"Rate of natural increase (per 1,000 population)",measure,
srb,Sex ratio at birth (male b

In [48]:
peroidInd['VarID'].unique()

array([2])

In [49]:
peroidInd['MidPeriod'].unique()

array([1953, 1958, 1963, 1968, 1973, 1978, 1983, 1988, 1993, 1998, 2003,
       2008, 2013, 2018, 2023, 2028, 2033, 2038, 2043, 2048, 2053, 2058,
       2063, 2068, 2073, 2078, 2083, 2088, 2093, 2098])

In [15]:
peroidInd_cty = peroidInd[peroidInd['LocID'].isin(countries.country)].drop(['Location', 'VarID', 'Variant', 'MidPeriod'], axis=1).copy()

In [16]:
peroidInd_cty['time'] = peroidInd_cty['Time'].map(lambda x: x.split('-')[0])

In [17]:
peroidInd_cty = peroidInd_cty.drop('Time', axis=1).set_index(['LocID', 'time'])

In [59]:
peroidInd_cty.head()

TFR    NRR     CBR    Births    LEx  LExMale  LExFemale      IMR  \
LocID time                                                                      
4     1950  7.45  1.636  50.314  2015.476  28.61    27.94      29.43  275.866   
      1955  7.45  1.765  50.998  2201.589  31.13    30.43      31.97  253.647   
      1960  7.45  1.899  51.477  2439.130  33.74    32.99      34.64  230.194   
      1965  7.45  2.017  51.646  2728.221  36.15    35.37      37.06  211.072   
      1970  7.45  2.141  51.234  3056.451  38.74    37.92      39.67  191.642   

                 Q5     CDR    Deaths  DeathsMale  DeathsFemale   CNMR  \
LocID time                                                               
4     1950  405.090  36.862  1476.605     794.020       682.585 -0.499   
      1955  374.138  33.718  1455.605     783.110       672.495 -0.463   
      1960  341.957  30.808  1459.784     777.437       682.347 -0.422   
      1965  314.708  28.223  1490.896     790.745       700.151 -0.379   
      1970  286.630  25.495  1520.938     802.414       718.524 -0.335   

            NetMigrations  GrowthRate  NatIncr   SRB     MAC  
LocID time                                                    
4     1950          -20.0       1.296   13.452  1.06  29.835  
      1955          -20.0       1.683   17.280  1.06  29.835  
      1960          -20.0       2.026   20.669  1.06  29.835  
      1965          -20.0       2.307   23.423  1.06  29.835  
      1970          -20.0       2.544   25.739  1.06  29.835

In [18]:
peroidInd_cty.index.names = ['country', 'time']

In [87]:
deaths_sex = peroidInd_cty[['DeathsMale', 'DeathsFemale']].copy()

In [88]:
deaths_sex_dp = []

m = deaths_sex[['DeathsMale']].copy()
m.columns = ['deaths']
m['gender'] = 'male'
m['freq'] = '5yearly'
m = m.set_index(['gender', 'freq'], append=True)

f = deaths_sex[['DeathsFemale']].copy()
f.columns = ['deaths']
f['gender'] = 'female'
f['freq'] = '5yearly'
f = f.set_index(['gender', 'freq'], append=True)

deaths_sex_dp.append(m)
deaths_sex_dp.append(f)

In [89]:
deaths_sex_dp = pd.concat(deaths_sex_dp, sort=False).dropna()

In [90]:
deaths_sex_dp.to_csv('../../deaths--by--country--time--gender--freq/ddf--datapoints--deaths--by--country--time--gender--freq.csv')

In [77]:
deaths_sex = peroidInd_cty[['LExMale', 'LExFemale']].copy()

In [83]:
deaths_sex_dp = []

m = deaths_sex[['LExMale']].copy()
m.columns = ['lex']
m['gender'] = 'male'
m['freq'] = '5yearly'
m = m.set_index(['gender', 'freq'], append=True)

f = deaths_sex[['LExFemale']].copy()
f.columns = ['lex']
f['gender'] = 'female'
f['freq'] = '5yearly'
f = f.set_index(['gender', 'freq'], append=True)

deaths_sex_dp.append(m)
deaths_sex_dp.append(f)


In [84]:
deaths_sex_dp = pd.concat(deaths_sex_dp, sort=False).dropna()

In [85]:
deaths_sex_dp.shape

(12060, 1)

In [80]:
!mkdir ../../lex--by--country--time--gender--freq

In [86]:
deaths_sex_dp.to_csv('../../lex--by--country--time--gender--freq/ddf--datapoints--lex--by--country--time--gender--freq.csv')

In [19]:
# other period indicators
peroidInd_cty_2 = peroidInd_cty.drop(['DeathsMale', 'DeathsFemale', 'LExMale', 'LExFemale'], axis=1)

In [20]:
import os

def create_dir(c, by):
    p = '../../{}--by--{}'.format(c, '--'.join(by))
    os.makedirs(p, exist_ok=True)
    return p

In [22]:
for c in peroidInd_cty_2:
    c_ = to_concept_id(c)
    p = create_dir(c_, ['country', 'time', 'freq'])
    df = peroidInd_cty_2[[c]]
    df.columns = [c_]
    df = df.dropna()
    df['freq'] = '5yearly'
    df = df.set_index('freq', append=True)
    df.to_csv(os.path.join(p, 'ddf--datapoints--{}--by--{}.csv'.format(c_, '--'.join(['country', 'time', 'freq']))))

In [24]:
mort = pd.read_csv('../source/csv/WPP2019_LifeTable.csv', encoding='latin1')

In [25]:
mort.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,SexID,Sex,AgeGrp,AgeGrpStart,...,mx,qx,px,lx,dx,Lx,Sx,Tx,ex,ax
0,4,Afghanistan,2,Medium,1950-1955,1953,1,Male,0,0,...,0.363600,0.292374,0.707626,100000.0,29237,80411,0.661270,2793722.0,27.937,0.330000
1,4,Afghanistan,2,Medium,1950-1955,1953,1,Male,1-4,1,...,0.049542,0.175186,0.824814,70763.0,12397,250224,0.858630,2713311.0,38.344,1.352000
2,4,Afghanistan,2,Medium,1950-1955,1953,1,Male,5-9,5,...,0.009532,0.046362,0.953638,58366.0,2706,283893,0.963771,2463087.0,42.201,2.066877
3,4,Afghanistan,2,Medium,1950-1955,1953,1,Male,10-14,10,...,0.006807,0.033460,0.966540,55660.0,1862,273608,0.961842,2179194.0,39.152,2.480526
4,4,Afghanistan,2,Medium,1950-1955,1953,1,Male,15-19,15,...,0.009295,0.045470,0.954530,53798.0,2446,263168,0.945028,1905586.0,35.421,2.620526


In [26]:
mort.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'SexID',
       'Sex', 'AgeGrp', 'AgeGrpStart', 'AgeGrpSpan', 'mx', 'qx', 'px', 'lx',
       'dx', 'Lx', 'Sx', 'Tx', 'ex', 'ax'],
      dtype='object')

In [27]:
mort['AgeGrp'].unique()

array(['0', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100+'],
      dtype=object)

In [28]:
age5 = mort[['AgeGrp']].drop_duplicates()

In [29]:
age5.columns = ['name']
age5['age5yearinterval'] = age5['name'].map(to_concept_id)

In [30]:
age5[['age5yearinterval', 'name']].to_csv('../../ddf--entities--age5yearinterval.csv', index=False)

In [31]:
mort_concepts = {
    "mx": "Central death rate, nmx, for the age interval (x, x+n)",
    "qx": "Probability of dying (nqx), for an individual between age x and x+n",
    "px": "Probability of surviving, (npx), for an individual of age x to age x+n",
    "lx": "Number of survivors, (lx), at age (x) for 100000 births",
    "dx": "Number of deaths, (ndx), between ages x and x+n",
    "Lx": "Number of person-years lived, (nLx), between ages x and x+n",
    "Sx": "Survival ratio (nSx) corresponding to proportion of the life table population in age group (x, x+n) who are alive n year later",
    "Tx": "Person-years lived, (Tx), above age x",
    "ex": "Expectation of life (ex) at age x, i.e., average number of years lived subsequent to age x by those reaching age x",
    "ax": "Average number of years lived (nax) between ages x and x+n by those dying in the interval"
}

In [33]:
mort_concepts_rename = {
    "mx": "Central death rate",
    "qx": "Probability of dying",
    "px": "Probability of surviving",
    "lx": "Number of survivors",
    "dx": "Number of deaths",
    "Lx": "Number of person-years lived",
    "Sx": "Survival ratio",
    "Tx": "Person-years lived",
    "ex": "Expectation of life",
    "ax": "Average number of years lived"
}

mort_concepts_rename = dict([(k, to_concept_id(v)) for k, v in mort_concepts_rename.items()])

In [34]:
mort_concepts_rename

{'mx': 'central_death_rate',
 'qx': 'probability_of_dying',
 'px': 'probability_of_surviving',
 'lx': 'number_of_survivors',
 'dx': 'number_of_deaths',
 'Lx': 'number_of_person_years_lived',
 'Sx': 'survival_ratio',
 'Tx': 'person_years_lived',
 'ex': 'expectation_of_life',
 'ax': 'average_number_of_years_lived'}

In [35]:
mort_concepts = dict([(mort_concepts_rename[k], v) for k, v in mort_concepts.items()])

In [36]:
mort_concepts

{'central_death_rate': 'Central death rate, nmx, for the age interval (x, x+n)',
 'probability_of_dying': 'Probability of dying (nqx), for an individual between age x and x+n',
 'probability_of_surviving': 'Probability of surviving, (npx), for an individual of age x to age x+n',
 'number_of_survivors': 'Number of survivors, (lx), at age (x) for 100000 births',
 'number_of_deaths': 'Number of deaths, (ndx), between ages x and x+n',
 'number_of_person_years_lived': 'Number of person-years lived, (nLx), between ages x and x+n',
 'survival_ratio': 'Survival ratio (nSx) corresponding to proportion of the life table population in age group (x, x+n) who are alive n year later',
 'person_years_lived': 'Person-years lived, (Tx), above age x',
 'expectation_of_life': 'Expectation of life (ex) at age x, i.e., average number of years lived subsequent to age x by those reaching age x',
 'average_number_of_years_lived': 'Average number of years lived (nax) between ages x and x+n by those dying in th

In [37]:
mort_concepts_df = pd.DataFrame.from_dict(mort_concepts, orient='index')
mort_concepts_df['concept_type'] = 'measure'
mort_concepts_df['domain'] = ''

print(mort_concepts_df.to_csv())

,0,concept_type,domain
central_death_rate,"Central death rate, nmx, for the age interval (x, x+n)",measure,
probability_of_dying,"Probability of dying (nqx), for an individual between age x and x+n",measure,
probability_of_surviving,"Probability of surviving, (npx), for an individual of age x to age x+n",measure,
number_of_survivors,"Number of survivors, (lx), at age (x) for 100000 births",measure,
number_of_deaths,"Number of deaths, (ndx), between ages x and x+n",measure,
number_of_person_years_lived,"Number of person-years lived, (nLx), between ages x and x+n",measure,
survival_ratio,"Survival ratio (nSx) corresponding to proportion of the life table population in age group (x, x+n) who are alive n year later",measure,
person_years_lived,"Person-years lived, (Tx), above age x",measure,
expectation_of_life,"Expectation of life (ex) at age x, i.e., average number of years lived subsequent to age x by those reaching age x",measure,
average_number_of_years_lived,Average number of years l

In [ ]:
mort_df = mort[['LocID', 'Time', 'S']]

In [39]:
mort['VarID'].unique()

array([2])

In [40]:
mort.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'SexID',
       'Sex', 'AgeGrp', 'AgeGrpStart', 'AgeGrpSpan', 'mx', 'qx', 'px', 'lx',
       'dx', 'Lx', 'Sx', 'Tx', 'ex', 'ax'],
      dtype='object')

In [41]:
mort['Sex'].unique()

array(['Male', 'Female', 'Total'], dtype=object)

In [42]:
mort['SexID'].unique()

array([1, 2, 3])

In [43]:
mort_total = mort[mort['Sex'] == 'Total'].copy()

In [44]:
mort_total = mort_total[['LocID', 'Time', 'AgeGrp', 'mx', 'qx', 'px', 'lx',
                         'dx', 'Lx', 'Sx', 'Tx', 'ex', 'ax']]

In [45]:
mort_total = mort_total.rename(columns={'LocID': 'country', 'Time': 'time', 'AgeGrp': 'age5yearinterval'}).rename(columns=mort_concepts_rename)

In [46]:
mort_total = mort_total[mort_total['country'].isin(countries.country)]
mort_total['age5yearinterval'] = mort_total['age5yearinterval'].map(to_concept_id)
mort_total['time'] = mort_total['time'].map(lambda x: x.split('-')[0])

In [47]:
mort_total = mort_total.set_index(['country', 'time', 'age5yearinterval'])

In [50]:
for c in mort_total:
    by = mort_total.index.names
    p = create_dir(c, by=by)
    mort_total[[c]].to_csv(os.path.join(p, 'ddf--datapoints--{}--by--{}'.format(c, '--'.join(by))))